# BASIC MODEL

## Classes

In [1]:
#import libraries
import numpy as np
import pandas as pd
import pylab as plt
import time
import glob, os, os.path
import osmnx as ox
import networkx as nx
import cv2

In [2]:
class Ambulances(object):
  """An ambulance class"""
  count = 0 
  ambs = []
  def __init__(self,verb=False):
    self.a_uid = self.count #unique id
    self.a_home = None #initial location
    self.a_dead = False #to check if it is out of the model
    self.a_state = 0 #code for state=> e.g 0:wait; 1:moving
    self.a_origin = None #start location
    self.a_destination = None #end location
    self.a_velocity = (0,1) #needs to be adjusted based on unit of model
    #to control uid and an aggregated list
    self.__class__.count += 1
    Ambulances.ambs.append(self)


In [3]:
class Pedestrians(object):
  """A pedestrian class"""
  count = 0
  peds = []
  def __init__(self,verb=False):
    self.p_uid = self.count
    self.p_age = 0
    self.p_gender = 0 #0:female;1:male
    self.p_home = None
    self.p_triage = 0 #0:no level; 1:green; 2:yellow; 3:red; 4:black
    self.p_state = 0
    self.p_origin = None
    self.p_destination = None
    self.p_velocity = (0,0.5)
    #to control uid and an aggregated list
    self.__class__.count +=1
    Pedestrians.peds.append(self)

In [4]:
class Hospitals(object):
  """A hospital class"""
  count = 0
  bldgs = []
  def __init__(self,verb=False):
    self.b_uid = self.count
    self.b_capacity = 0 #a number of max in-patients
    self.b_home = None #location of the building
    self.b_state = 0 #situation of behavior e.g. 0:available; 1:unavailable
    self.b_staff = 0 #number of doctors
    self.b_damage = 0 #level or flag of damage 0 is none
    self.b_inpatients = 0 #number of patients
    #to control uid and an aggregated list
    self.__class__.count +=1
    Hospitals.bldgs.append(self)

In [5]:
class Environment(object):
  def __init__(self,bbox,verb=False):
    #bbox is a dict of north,south,east,west lat,lon edges of target area
    self.e_bbox = {'north': bbox['north'], 'south': bbox['south'],
                     'east': bbox['east'], 'west': bbox['west']}
    self.e_network(self)

  def e_network(self,verb=False):
    # Obtain the roadmap data from OpenStreetMap by using OSMNX 
    self.e_G = ox.graph_from_bbox(self.e_bbox['north'], self.e_bbox['south'], 
                           self.e_bbox['east'], self.e_bbox['west'], 
                           network_type='drive')
  
  def e_project_network(self,verb=False):
    #Project network
    self.e_Gp = ox.project_graph(self.e_G)
  
  def e_get_nodes(self,verb=False):
    self.e_nodes, self.e_edges = ox.graph_to_gdfs(self.e_G)
    return self.e_nodes
  
  def e_get_edges(self,verb=False):
    self.e_nodes, self.e_edges = ox.graph_to_gdfs(self.e_G)
    return self.e_edges

  def e_plot(self,verb=False):
    #returning a fig and ax
    return ox.plot_graph(self.e_G,figsize=(16,8),bgcolor='w',node_color='k',
                         node_alpha=0.1,edge_color=(0,0,0,0.5),show=False,
                         close=False)

In [6]:
class Model(object):
  #Default parameters: Area (Kochi)
  arahama = {'north': 38.2271, 'south': 38.2077,
           'east': 140.9894, 'west': 140.9695}
  def __init__(self,bbox=arahama,hospitals=10,ambulances=3,
               population=50,verb=False):
    self.Env = Environment(bbox)
    self.nodes = self.Env.e_get_nodes()
    self.edges = self.Env.e_edges
    for i in range(hospitals):
      h = Hospitals()
      s = self.nodes.sample()
      h.home = (float(s['x']),
                float(s['y']))
    for i in range(ambulances):
      a = Ambulances()
      s = self.nodes.sample()
      a.home = (float(s['x']),
                float(s['y']))
    for i in range(population):
      p = Pedestrians()
      s = self.nodes.sample()
      p.home = (float(s['x']),
                float(s['y']))

  def go(self,sim_time=5,plot=False,video=False,verb=False):
    for t in range(sim_time):
      for i,a in enumerate(Ambulances.ambs):
        s = self.nodes.sample()
        a.home = (float(s['x']),
                float(s['y']))
      if plot:
        self.plot(id=t,save=True)
    if video:
      self.video()

  def plot(self,id=0,save=False,verb=False):
    #plot network
    fig,ax = self.Env.e_plot()
    #plot hospitals
    x = [h.home[0] for h in Hospitals.bldgs]
    y = [h.home[1] for h in Hospitals.bldgs]
    ax.scatter(x,y,c='r',marker='+')
    #plot ambulances
    x = [a.home[0] for a in Ambulances.ambs]
    y = [a.home[1] for a in Ambulances.ambs]
    ax.scatter(x,y,c='b',marker='s')
    #plot pedestrians
    x = [p.home[0] for p in Pedestrians.peds]
    y = [p.home[1] for p in Pedestrians.peds]
    ax.scatter(x,y,c='m',marker='.')
    if save:
      plt.savefig(f'/content/drive/MyDrive/Colab Notebooks/img/fig{id:04d}.png')
      plt.close()
      print(f'figure {id:04d} plotted')
  
  def video(self,image_folder='/content/drive/MyDrive/Colab Notebooks/img',
            video_name = '/content/drive/MyDrive/Colab Notebooks/anim.avi',
            fps=1,verbose=False):
    images = [img for img in sorted(os.listdir(image_folder)) if img.endswith(".png")]
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape
    video = cv2.VideoWriter(video_name,cv2.VideoWriter_fourcc('M','J','P','G'), 
                            fps, (width,height))
    for image in images:
        if verbose:
            print(image)
        video.write(cv2.imread(os.path.join(image_folder, image)))
    cv2.destroyAllWindows()
    video.release()

## Parameters

For bounding box (bbox) of other areas:
[OSM](https://www.openstreetmap.org/export#map=5/33.907/138.460)

In [7]:
#BBOX
#small area is faster (by default this is in the class)
arahama = {'north': 38.2271, 'south': 38.2077,
           'east': 140.9894, 'west': 140.9695}

#this is the same extension from Abe san's simulation
kochi = {'north': 33.5978428707312631, 'south': 33.3844862625877710,
           'east': 133.7029719124942346, 'west': 133.3254475395832799}

## Running the model

In [8]:
#create a model class of an area with number of hospitals, ambulances and population
t = time.time()
M = Model(bbox=kochi,hospitals=10,ambulances=10,
               population=50,verb=False)
print(time.time()-t)

48.508816957473755


In [9]:
#plot current situation ('id' is an integer)
M.plot(id=0,save=True)

figure 0000 plotted


In [10]:
#number of steps for simulation, can plot and make video same time
t = time.time()
M.go(sim_time=10,plot=True,video=True)
print(time.time()-t)

figure 0000 plotted
figure 0001 plotted
figure 0002 plotted
figure 0003 plotted
figure 0004 plotted
figure 0005 plotted
figure 0006 plotted
figure 0007 plotted
figure 0008 plotted
figure 0009 plotted
113.93774580955505


In [11]:
#only to make video
M.video()